In [66]:
import os
import sys
import numpy as np
import neo
import pandas as pd
import h5py
import ast
import McsPy
import sys, importlib, os
import McsPy.McsData
import McsPy.McsCMOS
from McsPy import ureg, Q_
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz, find_peaks, correlate, gaussian, filtfilt
from scipy import stats
from scipy import signal
from scipy import stats
from scipy import signal
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import McsPy
import McsPy.McsData
from McsPy import ureg, Q_
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import scipy
import time
import glob
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import seaborn as sns
import copy
import pickle
import fnmatch

# Plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns
#networkx
import plotly.graph_objects as go
import networkx as nx
import matplotlib.patches as mpatche


%matplotlib widget

In [67]:
working_directory = '/Users/jonas/Documents/DATA/MEA_DATA_Aachen_sample/output/2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1'

In [68]:
os.chdir(working_directory)

In [69]:
filename=working_directory.split('/')[-1]

In [70]:
glob.glob('*')

['interesting_burst_zoomed_10082021_b25.png',
 'divided_burst_spatial_zoomed_10082021_b25.png',
 'infodics_2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1.npy',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1_burst_starts_dictionary.npy',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1__raster_firingrate_plot.eps',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1raster_and_FR_plot_10082021_b25.png',
 'interesting_burst_10082021_b25.png',
 'spike_extraction',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1_full_spikedic.npy',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1__raster_firingrate_plot.pdf',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1whole_MEA_Heatmap.png',
 'interesting_burst_10082021_b21.png',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1__raster_firingrate_plot.png',
 '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1__raster_firingrate_plot.svg',
 'MAIN_RECORDING_Dictionary_2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1.pkl']

In [113]:
'MAIN_RECORDING_Dictionary_'+filename+'.pkl'

'MAIN_RECORDING_Dictionary_2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1.pkl'

In [114]:
MAIN_RECORDING_DICTIONARY = pickle.load(open('MAIN_RECORDING_Dictionary_'+filename+'.pkl', "rb"))

In [115]:
MAIN_RECORDING_DICTIONARY.keys()

dict_keys(['spikedic_MAD', 'network_bursts_seconds', 'fr_dic', 'isi_dictionary', 'isi_average_dic', 'isi_std_dic', 'number_rand_notrand_spikes_per_channel', 'rand_notrand_spikes_per_channel', 'Infos_Recording', 'Infos_Analysis', 'Infos_Anatomy', 'Basics', 'Bursts', 'Interburst-Intervals', 'bursting_time_per_channel', 'bursts_per_channel', 'burst_connections'])

In [116]:
spikedic_MAD = MAIN_RECORDING_DICTIONARY['spikedic_MAD']
tick = MAIN_RECORDING_DICTIONARY['Infos_Recording']['tick']
timelengthrecording_s = MAIN_RECORDING_DICTIONARY['Infos_Recording']['timelengthrecording_s']
info_dics_subrecordings = MAIN_RECORDING_DICTIONARY['Infos_Recording']['info_dics_subrecordings']
first_recording_timepoint = info_dics_subrecordings['0_to_120']['first_recording_timepoint']
scale_factor_for_second = MAIN_RECORDING_DICTIONARY['Infos_Recording']['scale_factor_for_second']
relevant_factor = timelengthrecording_s*0.05


In [117]:
def remove_artefact_spikes(spikes, recording_length, window=0.02):
    
    removing_times = []
    i = 0
    removed_spikes = {} 
    kicked_out = {}
    while i < timelengthrecording_s:
        #print(i)
        activechannels = 0
        for key in spikes:
            for s in spikes[key]:
                if i <= s*tick*scale_factor_for_second < (i + window):
                    activechannels += 1
        if activechannels >= 190:
            removing_times.append(i)
            print(i)
        i += window
    
    print(removing_times)
    for key in spikes:
        new_spikes = []
        kicked_out_list = []

        s_list_save = list(spikes[key])
        s_list = list(spikes[key])
        for s in s_list_save:
            for time in removing_times:
                if time <= s*tick*scale_factor_for_second < (time + window*4):
                    print('time :' + str(time))
                    print('spike :' + str(s*tick*scale_factor_for_second))
                    kicked_out_list.append(s)
                    s_list.remove(s)
                    print('removed spike from channel ' + key)
                #else:
                 #   new_spikes.append(s)
        removed_spikes[key] = s_list
        kicked_out[key] = kicked_out_list

    return removed_spikes, kicked_out

In [118]:
spikedic_MAD, kicked_out = remove_artefact_spikes(spikes=spikedic_MAD, recording_length=timelengthrecording_s)

62.54000000000418
62.840000000004224
113.35999999999459
118.97999999999347
[62.54000000000418, 62.840000000004224, 113.35999999999459, 118.97999999999347]
time :62.54000000000418
spike :62.546519999999994
removed spike from channel G13
time :62.54000000000418
spike :62.547439999999995
removed spike from channel G13
time :62.54000000000418
spike :62.552
removed spike from channel G13
time :62.840000000004224
spike :62.84872
removed spike from channel G13
time :113.35999999999459
spike :113.36063999999999
removed spike from channel G13
time :113.35999999999459
spike :113.36112
removed spike from channel G13
time :113.35999999999459
spike :113.36972
removed spike from channel G13
time :113.35999999999459
spike :113.3706
removed spike from channel G13
time :113.35999999999459
spike :113.37556
removed spike from channel G13
time :118.97999999999347
spike :118.99199999999999
removed spike from channel G13
time :118.97999999999347
spike :118.99275999999999
removed spike from channel G13
time 

In [119]:
spikedic_MAD['G13']

[439468,
 1453618,
 1911206,
 2181411,
 2259021,
 2259103,
 2259379,
 2259791,
 2665089,
 2665188,
 2665503,
 2665755]

In [120]:
kicked_out['G13']

[1563663,
 1563686,
 1563800,
 1571218,
 2834016,
 2834028,
 2834243,
 2834265,
 2834389,
 2974800,
 2974819,
 2974964]

In [121]:
# to use when only relevant channels shall be plotted
'''
active_channels = 0
spikedic_seconds = {}
for key in spikedic_MAD:
    relevant_factor = timelengthrecording_s*0.05
    if len(spikedic_MAD[key])>relevant_factor:
        sec_array = np.asarray(spikedic_MAD[key])*tick*scale_factor_for_second
        spikedic_seconds[key]=sec_array
        active_channels += 1
spikearray_seconds = np.asarray(list(spikedic_seconds.values()))  
'''

# to use when all channels shall be plotted
active_channels = 0
spikedic_seconds = {}
for key in spikedic_MAD:
    sec_array = np.asarray(spikedic_MAD[key])*tick*scale_factor_for_second
    spikedic_seconds[key]=sec_array
    active_channels += 1
spikearray_seconds = np.asarray(list(spikedic_seconds.values()))  


'\nactive_channels = 0\nspikedic_seconds = {}\nfor key in spikedic_MAD:\n    relevant_factor = timelengthrecording_s*0.05\n    if len(spikedic_MAD[key])>relevant_factor:\n        sec_array = np.asarray(spikedic_MAD[key])*tick*scale_factor_for_second\n        spikedic_seconds[key]=sec_array\n        active_channels += 1\nspikearray_seconds = np.asarray(list(spikedic_seconds.values()))  \n'

In [122]:
# for kicked out spikes

active_channels = 0
spikedic_seconds_kicked = {}
for key in kicked_out:
    sec_array = np.asarray(kicked_out[key])*tick*scale_factor_for_second
    spikedic_seconds_kicked[key]=sec_array
    active_channels += 1
spikearray_seconds_kicked = np.asarray(list(spikedic_seconds_kicked.values()))  

In [123]:
spikedic_seconds['G13']

array([ 17.57872,  58.14472,  76.44824,  87.25644,  90.36084,  90.36412,
        90.37516,  90.39164, 106.60356, 106.60752, 106.62012, 106.6302 ])

In [124]:
spikearray_seconds

array([array([ 17.57872,  58.14472,  76.44824,  87.25644,  90.36084,  90.36412,
               90.37516,  90.39164, 106.60356, 106.60752, 106.62012, 106.6302 ]),
       array([], dtype=float64), array([76.44784]),
       array([ 63.44876,  76.448  ,  87.25652,  90.33176,  90.34516,  90.35156,
               90.35428,  90.35916,  90.36308,  90.36544,  90.37072,  90.37472,
               90.37972,  90.38444,  90.39052,  90.392  ,  90.40056,  90.41052,
               90.49644,  90.49912,  90.5018 ,  90.53196,  90.5578 ,  90.56164,
               90.5648 ,  90.58936,  90.62692,  90.6292 ,  90.67216,  90.84996,
               90.85628, 106.5894 , 106.60288, 106.60616, 106.60712, 106.61136,
              106.6168 , 106.61896, 106.62908, 106.6356 , 106.6418 , 106.64748,
              106.66428, 106.68476, 106.6902 , 106.693  , 106.70096, 106.72456,
              106.76444, 106.78352, 106.83484, 106.8866 , 106.88952, 106.9332 ]),
       array([ 76.44772,  90.35444,  90.357  ,  90.35876,  90.36

In [125]:
type(spikearray_seconds[0])

numpy.ndarray

In [126]:
#ss = remove_artefact_spikes(spikes = spikearray_seconds, recording_length=timelengthrecording_s)

In [127]:
#t = [i.tolist() for i in spikearray_seconds]

In [128]:
len(spikedic_MAD['G13'])

12

In [129]:
relevant_factor

6.02

In [130]:
# get a 1-D array with every detected spike
scale_factor_for_milisecond = 1e-03
full_spike_list = []
full_spike_list_seconds = []
for key in spikedic_MAD:
    if len(spikedic_MAD[key])>relevant_factor:
        x = list(np.asarray(spikedic_MAD[key])*scale_factor_for_milisecond*tick)
        full_spike_list = full_spike_list + x

        xs = list(np.asarray(spikedic_MAD[key])*scale_factor_for_second*tick)
        full_spike_list_seconds = full_spike_list_seconds + xs
full_spikes = sorted(full_spike_list)
full_spikes_seconds = sorted(full_spike_list_seconds)

In [131]:
mean_fr_whole_recording = np.around((len(full_spikes_seconds) / timelengthrecording_s), 3)

In [132]:
mean_fr_whole_recording

22.807

In [133]:
len(full_spikes_seconds), len(full_spikes)

(2746, 2746)

In [134]:
 #define bins 
binsize = 0.005 #seconds
#bins= np.arange(0, timelengthrecording_s+binsize, binsize)

# make a histogram 
#full_spikes_binned = np.histogram(full_spikes_seconds, bins)[0]


#trial of population burst plot as inspired by Andrea Corna
bins = int(timelengthrecording_s / binsize)+1

firing_rate_histogram = np.histogram(full_spikes_seconds, bins=bins)
firing_rate = firing_rate_histogram[0]*200 #conversion to hertz
#firing_rate = firing_rate_histogram[0]

In [135]:
bins


24080

In [136]:
len(firing_rate_histogram[0])

24080

In [137]:
len(full_spikes_seconds)

2746

In [138]:
firing_rate_histogram

(array([2, 3, 9, ..., 4, 0, 3]),
 array([  0.7632    ,   0.76815068,   0.77310136, ..., 119.96565864,
        119.97060932, 119.97556   ]))

In [139]:
firing_rate_histogram[0][:500]


array([2, 3, 9, 5, 5, 3, 2, 0, 0, 1, 0, 0, 5, 9, 2, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [140]:
def gaussian_smoothing(y, window_size=10, sigma=500):

    filt = signal.gaussian(window_size, sigma)

    return signal.convolve(y, filt, mode='same')


In [141]:
# using firing rate histogram already conversed to hertz

N = int(1/binsize) # für eine Sekunde, das Sliding window, also letztlich number of bins
plot_N = int(0.01/binsize)
# gaussian smmothing fo the firing rate and moving average
fr_gau = gaussian_smoothing(firing_rate)
plot_fr_gau = gaussian_smoothing(firing_rate)

ma_fr_gau = np.convolve(fr_gau, np.ones(N)/N, mode='full')
plotting_ma_fr_gau = np.convolve(fr_gau, np.ones(plot_N)/plot_N, mode='full')


In [142]:
fr_gau[:300]

array([4799.90960127, 5399.93010089, 5799.9491006 , 5799.96910021,
       5799.96590022, 5999.93140083, 5599.90920118, 4999.88470188,
       4199.88430199, 4999.84230281, 4399.89300156, 3999.93800076,
       3599.97900013, 3599.98940008, 3599.98540018, 3399.9791001 ,
       3399.95110044, 3399.90950139, 2399.91480161,  599.97890039,
        199.99190016,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     

In [143]:
fig = plt.figure(figsize = (10,6))
gs = fig.add_gridspec(3, hspace = 0, height_ratios=[1,1,1])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black', linewidth=0.5)
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].plot(fr_gau, color = 'red', linewidth=0.5)
axs[2].plot(plotting_ma_fr_gau)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing Rate [Hz]')

In [144]:
# we look for the mean of the MA as threshold
# we arrange this mean in an array for plotting
mean_ma_fr_gau = np.mean(ma_fr_gau)
std_ma_fr_gau = np.std(ma_fr_gau)
network_burst_threshold = mean_ma_fr_gau
shape_for_threshold = np.shape(ma_fr_gau)
network_burst_threshold_array = np.full(shape_for_threshold, network_burst_threshold)
    

In [145]:
network_burst_threshold_array

array([225.61513067, 225.61513067, 225.61513067, ..., 225.61513067,
       225.61513067, 225.61513067])

In [146]:
#network_burst_threshold = 1*std_ma_fr_gau

In [147]:
# now we identify the burts from the network and will extract an array with 
# tuples containing the burst start and end times
bursts= []
burst_start = []
burst_seconds_start = []
burst_end = []
burst_seconds_end = []
for index in range(0, len(ma_fr_gau[:-N])):
    if ma_fr_gau[index+N] > network_burst_threshold:
        if ma_fr_gau[index+N-1] <= network_burst_threshold:
            burst_start.append(index)
        if index == 0:
            burst_start.append(0)
            #burst_seconds_start.append((index+N)*0.005)
    else:
        if (ma_fr_gau[index+N-1] > network_burst_threshold) and (len(burst_start)>0):
            if index+N > len(ma_fr_gau):
                ending = len(ma_fr_gau)
            else: 
                ending = index + N

            burst_end.append(ending)
            #burst_seconds_end.append((ending)*0.005)
bursts = list(zip(burst_start, burst_end))
    

In [148]:
for i in burst_start:
    burst_seconds_start.append(firing_rate_histogram[1][i])
for i in burst_end:
    if i >= len(firing_rate_histogram[1]):
        burst_seconds_end.append(firing_rate_histogram[1][-1])
    else:
        burst_seconds_end.append(firing_rate_histogram[1][i])

bursts_seconds = list(zip(burst_seconds_start, burst_seconds_end))
# bursts sind jetzt im 5ms bin   


In [149]:
def zoom_signal(fr, spiketimes, start_in_seconds, stop_in_seconds):
    
    '''
    for zooming in on the plot below
    '''
    
    start = start_in_seconds*200
    stop = stop_in_seconds*200
    
    cutted_fr_array = fr[start:stop+1]
    
    cutted_spiketimes = []
    for channel_spikes in spiketimes:
        s = [y for y in channel_spikes if stop_in_seconds > y >= start_in_seconds]
        cutted_spiketimes.append(s)
        
    return cutted_fr_array, cutted_spiketimes
        
        
        
        
    
    
    

In [150]:
a, b = zoom_signal(fr=fr_gau, spiketimes=spikearray_seconds, start_in_seconds=14, stop_in_seconds=30)

In [151]:
timelengthrecording_s

120.39999999999999

In [153]:
# logarhitmic y scale

fig = plt.figure(figsize = (10,6))
gs = fig.add_gridspec(3, hspace = 0, height_ratios=[1,1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(fr_gau, color= 'black', linewidth=0.5)
axs[0].set_ylabel('Firing Rate [Hz]')
#axs[0].set_yscale('symlog')
axs[0].set_ylim(0, 10000)
axs[1].set_ylim(0, 10000)
axs[1].plot(ma_fr_gau, color = 'black', linewidth=0.5)
axs[1].plot(network_burst_threshold_array, color='red', linewidth = 0.5)
axs[2].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5,
                 linelengths = 1, colors = 'black')
#axs[2].eventplot(spikearray_seconds_kicked, color = 'red', linewidths = 0.5,
 #                linelengths = 1, colors = 'red')
axs[2].set_ylabel('MEA Channels')

for ax in axs:
    for i in bursts_seconds:
        axs[2].axvspan(i[0], i[1], facecolor = '#5B89A6', alpha = 0.3)
#fig.savefig(filename+ '__raster_firingrate_plot.pdf', dpi=300)
#fig.savefig(filename+ '__raster_firingrate_plot.eps', dpi=300)
#fig.savefig(filename+ '__raster_firingrate_plot.svg', dpi=300)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing Rate [Hz]')

(0.0, 10000.0)

(0.0, 10000.0)

Text(0, 0.5, 'MEA Channels')

In [154]:
fig = plt.figure(figsize = (10,6))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[2,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black')
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5,
                 linelengths = 1, colors = 'black')
axs[1].set_ylabel('Relevant Channels')
fig.suptitle(filename)

#for ax in axs:
#    for i in bursts_seconds:
#        axs[1].axvspan(i[0], i[1], facecolor = '#5B89A6', alpha = 0.3)
#fig.savefig(filename+ '__raster_firingrate_plot_solo.png', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing Rate [Hz]')

Text(0, 0.5, 'Relevant Channels')

Text(0.5, 0.98, '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1')

In [155]:
# identify the longest burst and longest interburst-interval


In [156]:
#longest bursts:

longest_burst = 0
longest_burst_i = 0
for i in bursts_seconds:
    burstinterval = i[1]-i[0]
    if burstinterval > longest_burst:
        longest_burst = burstinterval
        longest_burst_i = i

In [157]:
# longest_interburst interval:

longest_ibi = 0
longest_ibi_times = (0, 0)

for i in range(1, len(bursts_seconds)-1):
    ibi = bursts_seconds[i+1][0] - bursts_seconds[i][1]
    if ibi > longest_ibi:
        longest_ibi = ibi
        longest_ibi_times = ((bursts_seconds[i][1]), (bursts_seconds[i+1][0])) 

In [158]:
longest_ibi_times

(65.15668698172757, 67.83005385880399)

In [159]:
fig = plt.figure(figsize = (20,8))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black')
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5, linelengths = 1)
axs[1].set_ylabel('Relevant Channels')

for ax in axs:
    for i in bursts_seconds:
        axs[1].axvspan(i[0], i[1], facecolor = '#118C8C', alpha = 0.3)
        axs[1].axvspan(longest_burst_i[0], longest_burst_i[1], facecolor = '#F2727D', alpha = 0.2, label = 'longest network burst')
        axs[1].axvspan(longest_ibi_times[0], longest_ibi_times[1], facecolor = '#D7D7D9', alpha = 0.1, ls =':')
        
#fig.savefig(filename+'raster_and_FR_plot.png', dpi=300)
        


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing Rate [Hz]')

Text(0, 0.5, 'Relevant Channels')

In [167]:
# next, we want to isolate these two areas for a spade analysis

In [177]:
def subdivide_spiketrain(spiketrain, sub_start = 0, sub_stop = 10, tick=tick, scale_factor_for_second=1e-06):
    '''
    Excpects: 
        a spiketrain with tick datapoints
        default ticks are 40
        default scale_factor_for_seconds = 1e-06
        provide the start and stop of the desired sub in seconds
    
    Does:
        converts the desired seconds into data ticks
        checks if the spikes of the given spiketrain is in the desired subs
        substracts the starting time -->
        
    Returns:
        a spiketrain dictionary that again starts from zero
    
    '''
    sub_start_tick = sub_start / (tick*scale_factor_for_second)
    sub_stop_tick = sub_stop / (tick*scale_factor_for_second)
    sub_spiketrain = {}
  
    for key in spiketrain: 
        list_per_key = []
        for i in spiketrain[key]:
            if (i>=sub_start_tick ) & (i<sub_stop_tick):
                list_per_key.append(int(i-sub_start_tick))
        sub_spiketrain[key]=list_per_key

    return sub_spiketrain

In [178]:
burst_seconds_start

[0.7632,
 3.530629785714286,
 7.154527107973422,
 11.332900523255814,
 16.06079935215947,
 19.43716270431894,
 23.189777691029903,
 26.774069578073092,
 31.279187833887047,
 35.87341831893688,
 37.62100814784053,
 41.79443088372093,
 46.75996232392026,
 50.98784253322259,
 55.05730100166113,
 58.74555715614618,
 62.706100677740864,
 67.83005385880399,
 71.97872319767441,
 75.46400149667774,
 80.42953293687708,
 84.97425662790698,
 88.51399240033223,
 94.08845740697674,
 98.34109101328903,
 103.76208495847176,
 105.57403361960132,
 110.03954644019933,
 114.35158819933555,
 118.94086800498339]

In [179]:
burst_seconds_end

[1.7879906362126246,
 6.1544898687707645,
 9.154601586378737,
 13.674571880398672,
 18.04107111295681,
 21.452089220930233,
 25.170049451827243,
 29.199902485049837,
 33.85849180232558,
 37.84873940033223,
 39.606230588039864,
 44.45294572259136,
 48.784790199335546,
 53.01762108803987,
 57.37421896179402,
 60.72582891694352,
 65.15668698172757,
 69.85983241362126,
 74.01840311129568,
 78.27598739700997,
 82.81080972923588,
 86.93472567109634,
 91.5091534385382,
 96.49943827574751,
 101.24713982225914,
 105.816616910299,
 107.70282576245847,
 112.34161236212624,
 116.37146539534884,
 119.97556]

In [180]:
st_longest_burst = subdivide_spiketrain(spikedic_MAD, longest_burst_i[0], longest_burst_i[1])

In [181]:
st_longest_ibi = subdivide_spiketrain(spikedic_MAD, longest_ibi_times[0], longest_ibi_times[1])

In [182]:
longest_burst

2.9951610382059783

In [165]:
# in the next step, we want to find the firing rates for every channel in hertz for these subdivisions

longest_burst_firingrate_dic = {}

for key in st_longest_burst:
    fr_channel = len(st_longest_burst[key])/longest_burst # i.e, number of spikes divided by duration -> results in number per second
    longest_burst_firingrate_dic[key] = fr_channel

In [166]:
# same is done for the longest interburst interval

longest_ibi_firingrate_dic = {}

for key in st_longest_ibi:
    fr_channel = len(st_longest_ibi[key])/longest_ibi # i.e, number of spikes divided by duration -> results in number per second
    longest_ibi_firingrate_dic[key] = fr_channel

In [167]:
# and lastly for the whole recording

whole_recording_firingrate_dic = {}

for key in spikedic_MAD:
    fr_channel = len(spikedic_MAD[key])/timelengthrecording_s # i.e, number of spikes divided by duration -> results in number per second
    whole_recording_firingrate_dic[key] = fr_channel

In [168]:
# next we plot this on a mea array:

mea_array=np.empty((16,16,))
mea_array[:]=np.nan
columnlist =['A','B','C','D','E','F','G','H','J','K','L','M','N','O','P','R']
df = pd.DataFrame(data=mea_array,columns=columnlist)
df.index+=1

df1 = pd.DataFrame(data=mea_array,columns=columnlist)
df1.index+=1

df2 = pd.DataFrame(data=mea_array,columns=columnlist)
df2.index+=1


In [169]:
# function to fill the dataframe with the 

def get_dict_values(df, dic):
    
    for i in df.columns:
        for j in range(0, 16):
            keystring = i+str(df.index[j])
            #print(keystring)
            if keystring in dic.keys():
                df.loc[df.index[j],i]=dic[keystring]
            
                
    
    return df

In [170]:
use_df_copy = df.copy()
use_df_copy1 = df1.copy()
use_df_copy2 = df2.copy()

In [171]:
#the dataframe for burst


df_fr_burst = get_dict_values(use_df_copy, longest_burst_firingrate_dic)

In [172]:
df_fr_burst

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.333872,0.000000,0.000000,0.000000,2.337103,0.333872,0.000000,0.000000,0.000000,NaN
2,0.0,0.0,0.333872,0.000000,0.000000,1.335487,0.333872,2.003231,4.674206,7.011309,3.338719,0.333872,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.000000,0.667744,0.000000,4.006462,0.000000,1.669359,0.333872,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.333872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333872,0.000000,0.333872,0.000000,0.000000,0.000000


In [173]:
# data frame for interburst interval
df_fr_ibi = get_dict_values(use_df_copy1, longest_ibi_firingrate_dic)

In [174]:
df_fr_burst

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.333872,0.000000,0.000000,0.000000,2.337103,0.333872,0.000000,0.000000,0.000000,NaN
2,0.0,0.0,0.333872,0.000000,0.000000,1.335487,0.333872,2.003231,4.674206,7.011309,3.338719,0.333872,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.000000,0.667744,0.000000,4.006462,0.000000,1.669359,0.333872,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.333872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333872,0.000000,0.333872,0.000000,0.000000,0.000000


In [199]:
f, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 4))

f.suptitle(filename)
sns.heatmap(df_fr_ibi, annot=False, linewidths=.2, ax=ax1, cmap="rocket_r", vmax=7)
ax1.set_title('Firing Rate per Channel - No Burst')


sns.heatmap(df_fr_burst, annot=False, linewidths=.2, ax=ax2, cmap="rocket_r", vmax=7)
ax2.set_title('Firing Rate per Channel - Burst')

#f.savefig(filename+'_MEA_Heatmap.png', dpi = 300)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, '2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1')

<AxesSubplot:>

Text(0.5, 1.0, 'Firing Rate per Channel - No Burst')

<AxesSubplot:>

Text(0.5, 1.0, 'Firing Rate per Channel - Burst')

In [200]:
df_fr_whole = get_dict_values(use_df_copy2, whole_recording_firingrate_dic)

In [201]:
df_fr_whole

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,0.008306,0.000000,0.000000,0.000000,0.008306,0.107973,0.257475,0.091362,0.124585,1.154485,0.058140,0.000000,0.000000,0.000000,NaN
2,0.008306,0.000000,0.107973,0.000000,0.000000,0.813953,0.440199,1.428571,2.126246,3.106312,2.616279,0.182724,0.000000,0.000000,0.000000,0.008306
3,0.016611,0.016611,0.000000,0.000000,0.041528,0.249169,0.049834,1.378738,0.357143,0.639535,0.099668,0.000000,0.008306,0.000000,0.000000,0.000000
4,0.016611,0.008306,0.008306,0.008306,0.008306,0.008306,0.008306,0.016611,0.000000,0.174419,0.000000,0.000000,0.000000,0.000000,0.000000,0.008306
5,0.008306,0.000000,0.000000,0.016611,0.000000,0.016611,0.008306,0.000000,0.000000,0.000000,0.008306,0.008306,0.000000,0.000000,0.016611,0.008306
6,0.000000,0.000000,0.000000,0.008306,0.000000,0.008306,0.000000,0.000000,0.000000,0.000000,0.008306,0.000000,0.008306,0.008306,0.016611,0.049834
7,0.008306,0.000000,0.008306,0.024917,0.000000,0.000000,0.016611,0.000000,0.008306,0.000000,0.008306,0.000000,0.000000,0.000000,0.008306,0.008306
8,0.008306,0.008306,0.000000,0.008306,0.024917,0.000000,0.008306,0.000000,0.000000,0.000000,0.008306,0.000000,0.000000,0.000000,0.000000,0.008306
9,0.008306,0.058140,0.008306,0.000000,0.008306,0.000000,0.008306,0.000000,0.016611,0.000000,0.000000,0.000000,0.000000,0.016611,0.008306,0.008306
10,0.008306,0.008306,0.008306,0.000000,0.008306,0.000000,0.000000,0.008306,0.000000,0.008306,0.041528,0.008306,0.041528,0.000000,0.000000,0.008306


In [202]:
'''
sum_mean_frs = 0
number_of_fr = 0
for key in whole_recording_firingrate_dic:
    if whole_recording_firingrate_dic[key] > 0:
        sum_mean_frs += whole_recording_firingrate_dic[key]
        number_of_fr += 1

mean_fr_whole_recording = np.around((sum_mean_frs/number_of_fr), 4)
'''

'\nsum_mean_frs = 0\nnumber_of_fr = 0\nfor key in whole_recording_firingrate_dic:\n    if whole_recording_firingrate_dic[key] > 0:\n        sum_mean_frs += whole_recording_firingrate_dic[key]\n        number_of_fr += 1\n\nmean_fr_whole_recording = np.around((sum_mean_frs/number_of_fr), 4)\n'

In [203]:
mean_fr_whole_recording

22.807

In [204]:
f, ax = plt.subplots(1, 1, figsize = (12,10))

sns.heatmap(df_fr_whole, annot=False, linewidths=.5, ax=ax, cmap="rocket_r")
ax.set_title('Fr Channel' + filename + ' Mean FR Active: {}, active channels: {}'.format(mean_fr_whole_recording, active_channels))

f.savefig(filename+'whole_MEA_Heatmap.png', dpi = 300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

Text(0.5, 1.0, 'Fr Channel2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1 Mean FR Active: 22.807, active channels: 252')

AttributeError: __delete__

In [56]:
np.save(filename+'sub_recording_BURST.npy', st_longest_burst)
np.save(filename+'sub_recording_NON-BURST.npy', st_longest_ibi)

In [57]:
filename

'2021-05-17_cortex_div11_aCSF_ID046_30µMNorepinephrine_spont_1_analyzed_on_03082021'

In [58]:
#overviewframe_ac = pd.DataFrame(columns=['aCSF', 'aCSF + 30µM Nor', 'rewash aCSF', 'hCSF', 'hCSF + CNQX'], index=['DIV8', 'DIV11'])

In [59]:
#overviewframe_fr = pd.DataFrame(columns=['aCSF', 'aCSF + 30µM Nor', 'rewash aCSF', 'hCSF', 'hCSF + CNQX'], index=['DIV8', 'DIV11'])

In [60]:
#overviewframe_fr.iloc[0, 4]=mean_fr_whole_recording

In [61]:
#overviewframe_ac.iloc[0, 4]=active_channels

In [62]:
#overviewframe_fr

In [63]:
overviewframe_ac

NameError: name 'overviewframe_ac' is not defined

In [64]:
longest_burst_i

(61.80625938870431, 64.49437591362125)

# Subdivision of a Burst of Interest

In the next part we take a Network Burst and plot it on a small scale to see how the spikes develop. 

Further, we will divide it in 200ms parts to see how the spikes will develop temporarily over the MEA Grid.

In [206]:
# we plot the beginning figure -> burst of interest is in yellow

# we choose the burst we want to plot and get start and end time in seconds
burst_of_interest = 26
b_of_interest_start = burst_seconds_start[burst_of_interest]
b_of_interest_end = burst_seconds_end[burst_of_interest]



fig = plt.figure(figsize = (10,4))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black')
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5, linelengths = 1)
axs[1].set_ylabel('Relevant Channels')

for ax in axs:
    for i in bursts_seconds:
        axs[1].axvspan(i[0], i[1], facecolor = '#118C8C', alpha = 0.3)
        
        axs[1].axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )
        
fig.savefig(filename+'raster_and_FR_plot_10082021_b25.png', dpi=300, bbox_inches = 'tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing Rate [Hz]')

Text(0, 0.5, 'Relevant Channels')

AttributeError: __delete__

In [207]:
# in the spikearray_seconds we have every spike used in the whole recording block
# we will now filter them to be in the time region of interest and create burst_of_interest_array

boi_array = []

for i in spikearray_seconds:
    channel_array_list = []
    for j in i:
        if (j >= b_of_interest_start) and (j <= b_of_interest_end):
            channel_array_list.append(j)
    channel_array = np.asarray(channel_array_list)
    boi_array.append(channel_array)

boi_array = np.asarray(boi_array)

In [208]:
# we can plot this subpart of the array

fig = plt.figure(figsize = (10,4))
gs = fig.add_gridspec(2, hspace = 0, height_ratios=[1,5])
axs = gs.subplots(sharex=False, sharey=False)
axs[0].plot(ma_fr_gau, color= 'black')
axs[0].set_ylabel('Firing Rate [Hz]')
axs[1].eventplot(spikearray_seconds, color = 'black', linewidths = 0.5, linelengths = 1)
axs[1].set_ylabel('Relevant Channels')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing Rate [Hz]')

Text(0, 0.5, 'Relevant Channels')

In [209]:
fig, ax = plt.subplots(1, 1, figsize = (10,4))

ax = plt.eventplot(boi_array, color = 'black', linewidths = 0.5, linelengths= 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [210]:
# we use the subdivide function to get the st of the burst of interest

st_burst_of_interest = subdivide_spiketrain(spikedic_MAD, b_of_interest_start, b_of_interest_end)

In [222]:
# we now want to make heatmaps to picture the activity to observe the spatial propagation

# define how much time of the st you want to observe
heatmap_window = 0.05 # in seconds

# make start and endpoints for each of these time-windows:

time_windows = []

# start of the subdivision = rounded start of the burst of interest
i = np.round(b_of_interest_start, 4)

# for every iteration, add the desired heatmap window
while i < b_of_interest_end:
    start = i
    stop = np.round(i+heatmap_window, 4)
    i = i + heatmap_window 
    if stop > b_of_interest_end:
        stop = b_of_interest_end
    time_windows.append((start, stop))


In [223]:
b_of_interest_start

105.57403361960132

In [224]:
time_windows

[(105.574, 105.624),
 (105.624, 105.674),
 (105.67399999999999, 105.724),
 (105.72399999999999, 105.774),
 (105.77399999999999, 105.824),
 (105.82399999999998, 105.874),
 (105.87399999999998, 105.924),
 (105.92399999999998, 105.974),
 (105.97399999999998, 106.024),
 (106.02399999999997, 106.074),
 (106.07399999999997, 106.124),
 (106.12399999999997, 106.174),
 (106.17399999999996, 106.224),
 (106.22399999999996, 106.274),
 (106.27399999999996, 106.324),
 (106.32399999999996, 106.374),
 (106.37399999999995, 106.424),
 (106.42399999999995, 106.474),
 (106.47399999999995, 106.524),
 (106.52399999999994, 106.574),
 (106.57399999999994, 106.624),
 (106.62399999999994, 106.674),
 (106.67399999999994, 106.724),
 (106.72399999999993, 106.774),
 (106.77399999999993, 106.824),
 (106.82399999999993, 106.874),
 (106.87399999999992, 106.924),
 (106.92399999999992, 106.974),
 (106.97399999999992, 107.024),
 (107.02399999999992, 107.074),
 (107.07399999999991, 107.124),
 (107.12399999999991, 107.174)

In [225]:
#time_windows = time_windows[3:]

In [226]:
fig, ax = plt.subplots(1, 1, figsize = (10,4))

ax = plt.eventplot(boi_array, color = 'black', linewidths = 0.5, linelengths= 2)
ax = plt.axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )

# plot the lines for the subdivisions
for i in time_windows:
    ax = plt.axvline(i[0])
    
    
#fig.savefig('interesting_burst_10082021_b21.png', dpi=300, bbox_inches = 'tight')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [227]:
# we can now
len(time_windows)

43

In [228]:
# we will now create a dictionary with key = starts of the subdivision
# and values = dictionary  of the corresponding spiketrains

subdivided_boi_st = {}
subdivided_boi_fr = {}

for i in time_windows:
    #empty dic for firing rates
    fr_subs = {}
    
    # get the spiketrains for the time subpart
    st = subdivide_spiketrain(spikedic_MAD, i[0], i[1])
    
    # assign the subdivided dictionary to the subdivision dictionary
    subdivided_boi_st[i[0]] = st
    
    # for firing rate, we calculate the fr for each channel
    for key in st:
        fr = len(st[key]) / (i[1]-i[0])
        
        #assign the firing rate to that key
        fr_subs[key] = fr
     
    # the value for the subdivision key is that used subdic, which is emptied in the next loop
    subdivided_boi_fr[i[0]] = fr_subs
        
    
    

In [229]:
# how many subplots we want to create
number_of_heatmaps = len(time_windows)

number_columns = int(number_of_heatmaps / 2)




In [230]:
# for plotting with sns heatmap we need a df first 

# this is done as empty df:

mea_array=np.empty((16,16,))
mea_array[:]=np.nan
columnlist =['A','B','C','D','E','F','G','H','J','K','L','M','N','O','P','R']
df = pd.DataFrame(data=mea_array,columns=columnlist)
df.index+=1


In [231]:
df

,A,B,C,D,E,F,G,H,J,K,L,M,N,O,P,R
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
fig = plt.figure(figsize=(20,8))

fig.tight_layout()
fig.suptitle('Burst Propagation from ' + 
             str(np.round(b_of_interest_start, 4))+ ' to ' + 
             str(np.round(b_of_interest_end, 4)))
gs1 = gridspec.GridSpec(2, 5)

#countries = ["Country " + str(i) for i in range(1, 26)]
axs = []
time_intra_burst = 0

for i in range(1, len(time_windows)+1):
    

    plotframe = copy.deepcopy(df)
    
    # get the key for the fr_dic from the time windows:
    key = time_windows[i-1][0] # substract one because we start the for range at 1
    
    #extract the corresponding fr_dic
    fr_dic = subdivided_boi_fr[key]
    
    # fill the dataframe
    plotframe_filled = get_dict_values(plotframe, fr_dic)
    
    axs.append(fig.add_subplot(gs1[i-1]))
    # plot the heatmap
    #sns.heatmap(plotframe_filled, annot=False, linewidths=.5, ax=axstring, cmap="YlOrRd")


    axs[-1] = sns.heatmap(plotframe_filled, 
                          annot=False, vmin=0, vmax = 20, cmap='flare', 
                          cbar = False, linewidths=0.1)
    
    
    plt.setp(axs[-1], 
             title=str(int(time_intra_burst)) + ' ms')
    
    time_intra_burst += (heatmap_window*1000)

fig.savefig('divided_burst_spatial_10082021_b25.png', dpi=300, bbox_inches = 'tight')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Burst Propagation from 105.574 to 107.7028')

[Text(0.5, 1.0, '0 ms')]

[Text(0.5, 1.0, '50 ms')]

[Text(0.5, 1.0, '100 ms')]

...

[Text(0.5, 1.0, '300 ms')]

[Text(0.5, 1.0, '350 ms')]

[Text(0.5, 1.0, '400 ms')]

[Text(0.5, 1.0, '450 ms')]

IndexError: index 10 is out of bounds for GridSpec with size 10

## we can try what happens if we zoom in further

In [233]:
b_of_interest_start = 106.5
b_of_interest_end = 106.85

In [234]:
fig, ax = plt.subplots(1, 1, figsize = (10,4))

ax = plt.eventplot(boi_array, color = 'black', linewidths = 0.5, linelengths= 2)
ax = plt.axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )

# plot the lines for the subdivisions
for i in time_windows:
    ax = plt.axvline(i[0])
    
    
fig.savefig('interesting_burst_10082021_b25.png', dpi=300, bbox_inches = 'tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: __delete__

In [235]:
# we now want to make heatmaps to picture the activity to observe the spatial propagation

# let us make the heatmap window generic, so the burst is always subdivied into 10 subplots
observed_time = b_of_interest_end-b_of_interest_start

heatmap_window = observed_time/10 # in seconds

# make start and endpoints for each of these time-windows:

time_windows = []

# start of the subdivision = rounded start of the burst of interest
i = np.round(b_of_interest_start, 4)

# for every iteration, add the desired heatmap window
while i < b_of_interest_end:
    start = i
    stop = np.round(i+heatmap_window, 4)
    i = i + heatmap_window 
    if stop > b_of_interest_end:
        stop = b_of_interest_end
    time_windows.append((start, stop))


In [236]:
time_windows

[(106.5, 106.535),
 (106.535, 106.57),
 (106.57, 106.605),
 (106.60499999999999, 106.64),
 (106.63999999999999, 106.675),
 (106.67499999999998, 106.71),
 (106.70999999999998, 106.745),
 (106.74499999999998, 106.78),
 (106.77999999999997, 106.815),
 (106.81499999999997, 106.85),
 (106.84999999999997, 106.85)]

In [237]:
# we again obtain the array of that burst
# in the spikearray_seconds we have every spike used in the whole recording block
# we will now filter them to be in the time region of interest and create burst_of_interest_array

boi_array = []

for i in spikearray_seconds:
    channel_array_list = []
    for j in i:
        if (j >= b_of_interest_start) and (j <= b_of_interest_end):
            channel_array_list.append(j)
    channel_array = np.asarray(channel_array_list)
    boi_array.append(channel_array)

boi_array = np.asarray(boi_array)

In [238]:
b_of_interest_end

106.85

In [239]:
fig, ax = plt.subplots(1, 1, figsize = (10,4))

ax = plt.eventplot(boi_array, color = 'black', linewidths = 0.5, linelengths= 2)
ax = plt.axvspan(b_of_interest_start, 
                       b_of_interest_end, facecolor = '#F2CD5E', alpha = 0.2, 
                       label = 'longest network burst'
                      )

# plot the lines for the subdivisions
for i in time_windows:
    ax = plt.axvline(i[0])
    
    
fig.savefig('interesting_burst_zoomed_10082021_b25.png', dpi=300, bbox_inches = 'tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: __delete__

In [240]:
# we will now create a dictionary with key = starts of the subdivision
# and values = dictionary  of the corresponding spiketrains

subdivided_boi_st = {}
subdivided_boi_fr = {}

for i in time_windows:
    #empty dic for firing rates
    fr_subs = {}
    
    # get the spiketrains for the time subpart
    st = subdivide_spiketrain(spikedic_MAD, i[0], i[1])
    
    # assign the subdivided dictionary to the subdivision dictionary
    subdivided_boi_st[i[0]] = st
    
    # for firing rate, we calculate the fr for each channel
    for key in st:
        fr = len(st[key]) / (i[1]-i[0])
        
        #assign the firing rate to that key
        fr_subs[key] = fr
     
    # the value for the subdivision key is that used subdic, which is emptied in the next loop
    subdivided_boi_fr[i[0]] = fr_subs
        

In [241]:
time_windows = time_windows[:10]

In [244]:
fig = plt.figure(figsize=(20,8))

fig.tight_layout()
fig.suptitle('Burst Propagation from ' + 
             str(np.round(b_of_interest_start, 4))+ ' to ' + 
             str(np.round(b_of_interest_end, 4)))
gs1 = gridspec.GridSpec(2, 5)

#countries = ["Country " + str(i) for i in range(1, 26)]
axs = []
time_intra_burst = 0

for i in range(1, len(time_windows)+1):
    

    plotframe = copy.deepcopy(df)
    
    # get the key for the fr_dic from the time windows:
    key = time_windows[i-1][0] # substract one because we start the for range at 1
    
    #extract the corresponding fr_dic
    fr_dic = subdivided_boi_fr[key]
    
    # fill the dataframe
    plotframe_filled = get_dict_values(plotframe, fr_dic)
    
    axs.append(fig.add_subplot(gs1[i-1]))
    # plot the heatmap
    #sns.heatmap(plotframe_filled, annot=False, linewidths=.5, ax=axstring, cmap="YlOrRd")


    axs[-1] = sns.heatmap(plotframe_filled, 
                          annot=False, vmin=0, cmap='rocket_r', 
                          cbar = False, linewidths=0.1)
    
    
    plt.setp(axs[-1], 
             title=str(int(time_intra_burst)) + ' ms')
    
    time_intra_burst += (heatmap_window*1000)

fig.savefig('divided_burst_spatial_zoomed_10082021_b25.png', dpi=300, bbox_inches = 'tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Burst Propagation from 106.5 to 106.85')

[Text(0.5, 1.0, '0 ms')]

[Text(0.5, 1.0, '34 ms')]

[Text(0.5, 1.0, '69 ms')]

...

[Text(0.5, 1.0, '209 ms')]

[Text(0.5, 1.0, '244 ms')]

[Text(0.5, 1.0, '279 ms')]

[Text(0.5, 1.0, '314 ms')]

AttributeError: __delete__